In [1]:
using Distributed
addprocs(10)

10-element Array{Int64,1}:
  2
  3
  4
  5
  6
  7
  8
  9
 10
 11

In [2]:
@everywhere using DelimitedFiles
@everywhere using Revise, MDToolbox
ENV["COLUMNS"]=110; #display width for MDToolbox

┌ Warning: no Manifest.toml file found, static paths used
└ @ Revise /opt/julia_depot/packages/Revise/moD4B/src/packagedef.jl:1361


In [3]:
receptor=readpdb("1cgi_receptor.pdb")
ligand=readpdb("1cgi_ligand.pdb")

1x440 TrjArray{Float64,Int64}
| I                          | I                          |  …   I                          |
| 1ASP                       | 1ASP                       |  …   56CYS                      |
| 1801N                      | 1802CA                     |  …   2240OXT                    |
|    64.86    34.03    29.79 |    65.13    34.96    28.68 |  …      51.32    20.79    29.47 |

In [4]:
receptor =  MDToolbox.set_radius(receptor)
ligand = MDToolbox.set_radius(ligand)

1x440 TrjArray{Float64,Int64}
| I                          | I                          |  …   I                          |
| 1ASP                       | 1ASP                       |  …   56CYS                      |
| 1801N                      | 1802CA                     |  …   2240OXT                    |
|    64.86    34.03    29.79 |    65.13    34.96    28.68 |  …      51.32    20.79    29.47 |

In [5]:
receptor_sasa = MDToolbox.compute_sasa(receptor, 1.4)
ligand_sasa = MDToolbox.compute_sasa(ligand, 1.4)

440-element Array{Float64,1}:
 33.14939845728653
 12.831049795424109
  4.528605810149686
 32.81343261192081
 25.158921167498253
  9.183006226136861
 35.71530080209068
 52.34523773806415
  4.898364720492511
  7.296087138574493
  4.528605810149686
 30.13478505176401
 20.378726145673582
  ⋮
  0.0
  5.357295120313601
 16.353298758873866
 26.416867225873165
 23.0204128682609
  0.0
  0.25158921167498255
  2.6416867225873166
 31.027667571816274
 15.346941912173934
  0.26808257310632905
 45.53700852266561

In [6]:
receptor = TrjArray(receptor, charge=receptor_sasa)
writepdb("test.pdb", receptor)

In [7]:
receptor = TrjArray(receptor, mass=receptor_sasa)
ligand = TrjArray(ligand, mass=ligand_sasa)

1x440 TrjArray{Float64,Int64}
| I                          | I                          |  …   I                          |
| 1ASP                       | 1ASP                       |  …   56CYS                      |
| 1801N                      | 1802CA                     |  …   2240OXT                    |
|    64.86    34.03    29.79 |    65.13    34.96    28.68 |  …      51.32    20.79    29.47 |

In [8]:
quaternions = readdlm("../data/quaternion/QUATERNION_LIST_576_Orient");
#quaternions = readdlm("../data/quaternion/QUATERNION_LIST_4608_Orient");
# quaternions = quaternions[1:10, :]
size(quaternions)
#quaternions = quaternions[1:50, :]

(576, 4)

In [9]:
grid_space = 1.2

1.2

In [10]:
@time result = MDToolbox.dock_fft(receptor, ligand, quaternions, grid_space=grid_space, tops=10)

Progress: 100%|█████████████████████████████████████████| Time: 0:01:07


162.545923 seconds (38.85 M allocations: 17.191 GiB, 1.62% gc time)


(receptor = 1x1799 TrjArray{Float64,Int64}
| E                          | E                          |  …   E                          |
| 1CYS                       | 1CYS                       |  …   245ASN                     |
| 1N                         | 2CA                        |  …   1799OXT                    |
|   -12.82    -4.83   -11.13 |   -12.18    -4.38    -9.89 |  …      -0.75     7.40   -21.12 |, ligand = 10x440 TrjArray{Float64,Int64}
| I                          | I                          |  …   I                          |
| 1ASP                       | 1ASP                       |  …   56CYS                      |
| 1801N                      | 1802CA                     |  …   2240OXT                    |
|   -27.79   -31.15   -35.48 |   -29.21   -30.92   -35.19 |  …     -14.48   -33.28   -22.17 |
|   -27.79   -29.95   -35.48 |   -29.21   -29.72   -35.19 |        -14.48   -32.08   -22.17 |
|   -27.79   -28.75   -35.48 |   -29.21   -28.52   -35.19 |        -14

In [11]:
result.score

10-element Array{Float64,1}:
 5.9816997236339375e-5
 5.981699359836057e-5
 5.899193638470024e-5
 5.8579429605742916e-5
 5.8579429605742916e-5
 5.8579429605742916e-5
 5.8579429605742916e-5
 5.8579429605742916e-5
 5.857942596776411e-5
 5.85794223297853e-5

In [12]:
result.ligand

10x440 TrjArray{Float64,Int64}
| I                          | I                          |  …   I                          |
| 1ASP                       | 1ASP                       |  …   56CYS                      |
| 1801N                      | 1802CA                     |  …   2240OXT                    |
|   -27.79   -31.15   -35.48 |   -29.21   -30.92   -35.19 |  …     -14.48   -33.28   -22.17 |
|   -27.79   -29.95   -35.48 |   -29.21   -29.72   -35.19 |        -14.48   -32.08   -22.17 |
|   -27.79   -28.75   -35.48 |   -29.21   -28.52   -35.19 |        -14.48   -30.88   -22.17 |
|   -28.99   -31.15   -31.88 |   -30.41   -30.92   -31.59 |        -15.68   -33.28   -18.57 |
|   -28.99   -28.75   -31.88 |   -30.41   -28.52   -31.59 |        -15.68   -30.88   -18.57 |
|   -28.99   -27.55   -31.88 |   -30.41   -27.32   -31.59 |  …     -15.68   -29.68   -18.57 |
|   -28.99   -28.75   -30.68 |   -30.41   -28.52   -30.39 |        -15.68   -30.88   -17.37 |
|   -28.99   -27.55   -30.68 

In [13]:
writepdb("result_receptor.pdb", result.receptor)
writepdb("result_ligand.pdb", result.ligand)